In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Code for OECD countries

In [2]:
oecd_countries = {'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'CAN': 'Canada', 'CHL': 'Chile', 'COL': 'Colombia', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece', 'HUN': 'Hungary', 'ISL': 'Iceland', 'IRL': 'Ireland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 'KOR': 'Korea', 'LVA': 'Latvia', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MEX': 'Mexico', 'NLD': 'Netherlands', 'NZL': 'New Zealand', 'NOR': 'Norway', 'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden', 'CHE': 'Switzerland', 'TUR': 'Turkey', 'GBR': 'United Kingdom', 'USA': 'United States'}

# Import data

## NEET

We select the neet rate corresponding to people aged between 15 and 29 years old, for OECD countries only and we select all the years available.

In [3]:
df = pd.read_csv('./data/education_system/NEET_men_women_altogether.csv')
df = df[(df.SUBJECT=='15_29')].drop(['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes'],axis=1)
df = df[(df.LOCATION).isin(oecd_countries.keys())]
df = df.reset_index(drop=True)

In [4]:
df.columns = ['Country','Time','NEET']

## Expenditure on LMP

### Prepare the dataframe

In [5]:
exp_lmp = pd.read_csv('./data/labour_market/public_exp_LMP.csv')
exp_lmp = exp_lmp[['LFS_COUNTRY','PROG','Time','Value']]
exp_lmp = exp_lmp[exp_lmp.PROG == 100].drop('PROG',axis=1).reset_index(drop=True)

In [6]:
exp_lmp.columns=['Country', 'Time', 'Exp_LMP']

### Add the values to the DF and create non-linear variables

In [7]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = exp_lmp[(exp_lmp.Country==country)&(exp_lmp.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Exp_LMP.values[0])
df['Exp_LMP']= list_value
df['Exp_LMP_2']=df['Exp_LMP']**2
df['Exp_LMP_3']=df['Exp_LMP']**3

## STR

We select only the STR for primary education ($L1$).

### Prepare the dataframe

In [8]:
STR = pd.read_csv('./data/education_system/student_teacher_ratio.csv')
STR = STR[['COUNTRY','ISC11_LEVEL_CAT','Year','Value']]
STR_L1 = STR[STR.ISC11_LEVEL_CAT == 'L1'].drop('ISC11_LEVEL_CAT',axis=1).reset_index(drop=True)

In [9]:
STR_L1.columns = ['Country', 'Time', 'STR']

### Add the values to the DF and create non-linear variables

In [10]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = STR_L1[(STR_L1.Country==country)&(STR_L1.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.STR.values[0])
df['STR']= list_value
df['STR_2']=df['STR']**2
df['STR_3']=df['STR']**3

In [11]:
# drop NaN values in order to remove useless observations from the dataframe
df=df.dropna()

In [12]:
df

,Country,Time,NEET,Exp_LMP,Exp_LMP_2,Exp_LMP_3,STR,STR_2,STR_3
16,AUS,2013,13.015899,0.87,0.7569,0.658503,15.615,243.828225,3807.377733
17,AUS,2014,12.647472,0.93,0.8649,0.804357,15.612,243.734544,3805.183701
18,AUS,2015,11.831610,0.91,0.8281,0.753571,15.433,238.177489,3675.793188
19,AUS,2016,11.352150,0.86,0.7396,0.636056,15.168,230.068224,3489.674822
20,AUS,2017,10.946128,0.85,0.7225,0.614125,15.124,228.735376,3459.393827
...,...,...,...,...,...,...,...,...,...
690,LTU,2014,14.185811,0.43,0.1849,0.079507,10.233,104.714289,1071.541319
691,LTU,2015,13.743647,0.53,0.2809,0.148877,10.256,105.185536,1078.782857
692,LTU,2016,11.403278,0.51,0.2601,0.132651,10.470,109.620900,1147.730823
693,LTU,2017,11.220660,0.54,0.2916,0.157464,10.619,112.763161,1197.432007


## Economic features

### GDP and LogGDP

In [13]:
gdp_per_person = pd.read_csv('./data/economic_features/GDP_per_capita.csv', sep=';')
gdp_per_person_oecd = pd.DataFrame(columns=gdp_per_person.columns)
for code in oecd_countries.keys():
    gdp_per_person_oecd = gdp_per_person_oecd.append(gdp_per_person[gdp_per_person['Country Code']==code])
gdp_2015_oecd = gdp_per_person_oecd[['Country Code', '2015']]
gdp_2017_oecd = gdp_per_person_oecd[['Country Code', '2017']]
gdp_1960_2019_oecd_full = gdp_per_person_oecd.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis=1).reset_index(drop=True)

gdp_1960_2019_oecd = pd.DataFrame(columns=['Country', 'Time', 'GDP'])
for i in range(len(gdp_1960_2019_oecd_full)):
    country = gdp_1960_2019_oecd_full.iloc[i][0]
    for j in range(1,len(gdp_1960_2019_oecd_full.columns)):
        year = gdp_1960_2019_oecd_full.columns[j]
        gdp = gdp_1960_2019_oecd_full.iloc[i][j]
        gdp_1960_2019_oecd = gdp_1960_2019_oecd.append({'Country': country, 'Time': int(year), 'GDP': gdp}, ignore_index=True)
    gdp_1960_2019_oecd = gdp_1960_2019_oecd.dropna()

In [14]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = gdp_1960_2019_oecd[(gdp_1960_2019_oecd.Country==country)&(gdp_1960_2019_oecd.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.GDP.values[0])
df['GDP']= list_value
df['LogGDP']=np.log(df['GDP'])

### CPI

In [15]:
cpi_2000_2019 = pd.read_csv('./data/economic_features/CPI_2000_2019.csv')
cpi_2000_2019 = cpi_2000_2019[['LOCATION', 'TIME', 'Value']]
cpi_2000_2019.columns = ['Country', 'Time', 'CPI']

In [16]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = cpi_2000_2019[(cpi_2000_2019.Country==country)&(cpi_2000_2019.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.CPI.values[0])
df['CPI']= list_value

### DEBT

In [17]:
debt_2000_2019 = pd.read_csv('./data/economic_features/DEBT_2000_2019.csv')
debt_2000_2019 = debt_2000_2019[['LOCATION', 'TIME', 'Value']]
debt_2000_2019.columns = ['Country', 'Time', 'DEBT']

In [18]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = debt_2000_2019[(debt_2000_2019.Country==country)&(debt_2000_2019.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.DEBT.values[0])
df['DEBT']= list_value

In [19]:
df

,Country,Time,NEET,Exp_LMP,Exp_LMP_2,Exp_LMP_3,STR,STR_2,STR_3,GDP,LogGDP,CPI,DEBT
16,AUS,2013,13.015899,0.87,0.7569,0.658503,15.615,243.828225,3807.377733,68150.107041,11.129468,2.449889,55.70968
17,AUS,2014,12.647472,0.93,0.8649,0.804357,15.612,243.734544,3805.183701,62510.791171,11.043094,2.487923,61.36842
18,AUS,2015,11.831610,0.91,0.8281,0.753571,15.433,238.177489,3675.793188,56755.721712,10.946512,1.508367,64.18005
19,AUS,2016,11.352150,0.86,0.7396,0.636056,15.168,230.068224,3489.674822,49971.131456,10.819201,1.276991,68.39156
20,AUS,2017,10.946128,0.85,0.7225,0.614125,15.124,228.735376,3459.393827,54027.966818,10.897257,1.948647,65.60463
...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,LTU,2014,14.185811,0.43,0.1849,0.079507,10.233,104.714289,1071.541319,16564.956549,9.715045,0.103758,52.58745
691,LTU,2015,13.743647,0.53,0.2809,0.148877,10.256,105.185536,1078.782857,14258.229335,9.565090,-0.884097,53.33852
692,LTU,2016,11.403278,0.51,0.2601,0.132651,10.470,109.620900,1147.730823,14998.125060,9.615680,0.905525,50.88945
693,LTU,2017,11.220660,0.54,0.2916,0.157464,10.619,112.763161,1197.432007,16885.407395,9.734205,3.722889,47.00182


## Education features

### Years of schooling

In [20]:
years_schooling = pd.read_csv('./data/education_system/mean-years-of-schooling-world.csv')
years_schooling=years_schooling.drop('Entity',axis=1)
years_schooling.columns=['Country', 'Time', 'Years_schooling']

In [21]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = years_schooling[(years_schooling.Country==country)&(years_schooling.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Years_schooling.values[0])
df['Years_schooling']= list_value

### Expenditure on education

In [22]:
spendings_educ = pd.read_csv('./data/education_system/spendings_in_education.csv')
spendings_educ = spendings_educ[(spendings_educ.ISC11=='L1')& (spendings_educ.EXPENDITURE_TYPE=='T')]
spendings_educ = spendings_educ[['COUNTRY', 'YEAR','Value']]
spendings_educ.columns = ['Country', 'Time', 'Exp_educ']

In [23]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = spendings_educ[(spendings_educ.Country==country)&(spendings_educ.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Exp_educ.values[0])
df['Exp_educ']= list_value

df['LogExp_educ']= np.log(df['Exp_educ'],where=~np.isnan(np.array(df['Exp_educ'])))

In [24]:
df

,Country,Time,NEET,Exp_LMP,Exp_LMP_2,Exp_LMP_3,STR,STR_2,STR_3,GDP,LogGDP,CPI,DEBT,Years_schooling,Exp_educ,LogExp_educ
16,AUS,2013,13.015899,0.87,0.7569,0.658503,15.615,243.828225,3807.377733,68150.107041,11.129468,2.449889,55.70968,12.6,9241.9922,9.131513
17,AUS,2014,12.647472,0.93,0.8649,0.804357,15.612,243.734544,3805.183701,62510.791171,11.043094,2.487923,61.36842,12.7,9257.9980,9.133243
18,AUS,2015,11.831610,0.91,0.8281,0.753571,15.433,238.177489,3675.793188,56755.721712,10.946512,1.508367,64.18005,12.8,9524.7178,9.161646
19,AUS,2016,11.352150,0.86,0.7396,0.636056,15.168,230.068224,3489.674822,49971.131456,10.819201,1.276991,68.39156,12.9,10022.5670,9.212595
20,AUS,2017,10.946128,0.85,0.7225,0.614125,15.124,228.735376,3459.393827,54027.966818,10.897257,1.948647,65.60463,12.9,10238.4130,9.233902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,LTU,2014,14.185811,0.43,0.1849,0.079507,10.233,104.714289,1071.541319,16564.956549,9.715045,0.103758,52.58745,13.0,5288.8008,8.573347
691,LTU,2015,13.743647,0.53,0.2809,0.148877,10.256,105.185536,1078.782857,14258.229335,9.565090,-0.884097,53.33852,13.0,5523.3896,8.616747
692,LTU,2016,11.403278,0.51,0.2601,0.132651,10.470,109.620900,1147.730823,14998.125060,9.615680,0.905525,50.88945,13.0,6170.3911,8.727518
693,LTU,2017,11.220660,0.54,0.2916,0.157464,10.619,112.763161,1197.432007,16885.407395,9.734205,3.722889,47.00182,13.0,6339.7036,8.754587


## Labour indicators

### Protection of workers

In [25]:
strictness = pd.read_csv('./data/labour_market/strictness of employment protection from 1998 to 2018.csv')
strictness = strictness[(strictness.SERIES == 'EPRC_V2')]
strictness = strictness[['COUNTRY','TIME','Value']]
strictness.columns = ['Country', 'Time', 'Strictness_of_workers']

In [26]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = strictness[(strictness.Country==country)&(strictness.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Strictness_of_workers.values[0])
df['Strictness_of_workers']= list_value

### Percentage of Part time employement

In [27]:
ft_pt_employ = pd.read_csv('./data/labour_market/percentage of full time part time employment.csv')
ft_pt_employ = ft_pt_employ[(ft_pt_employ.AGE==1524)&(ft_pt_employ.EMPSTAT=='TE')&(ft_pt_employ.SEX=='MW')]
ft_pt_employ = ft_pt_employ[['COUNTRY','SERIES','Time','Value']]
ft_pt_employ.columns = ['Country','Series','Time','Value']

pt_employ = ft_pt_employ[ft_pt_employ.Series=='PT'].drop('Series',axis=1)

In [28]:
list_value = []
for el in df.itertuples():
    country, time = el[1], el[2]
    tmp = pt_employ[(pt_employ.Country == country) & (pt_employ.Time == time)]
    if len(tmp.index) == 0:
        list_value.append(None)
    else:
        list_value.append(tmp.Value.values[0])
df['PT_employ'] = list_value

## Add a constant for the OLS regressions and save the df in a csv file

In [29]:
df = sm.add_constant(df)

In [30]:
df

,const,Country,Time,NEET,Exp_LMP,Exp_LMP_2,Exp_LMP_3,STR,STR_2,STR_3,GDP,LogGDP,CPI,DEBT,Years_schooling,Exp_educ,LogExp_educ,Strictness_of_workers,PT_employ
16,1.0,AUS,2013,13.015899,0.87,0.7569,0.658503,15.615,243.828225,3807.377733,68150.107041,11.129468,2.449889,55.70968,12.6,9241.9922,9.131513,2.011,46.205162
17,1.0,AUS,2014,12.647472,0.93,0.8649,0.804357,15.612,243.734544,3805.183701,62510.791171,11.043094,2.487923,61.36842,12.7,9257.9980,9.133243,2.011,46.603383
18,1.0,AUS,2015,11.831610,0.91,0.8281,0.753571,15.433,238.177489,3675.793188,56755.721712,10.946512,1.508367,64.18005,12.8,9524.7178,9.161646,2.011,47.087093
19,1.0,AUS,2016,11.352150,0.86,0.7396,0.636056,15.168,230.068224,3489.674822,49971.131456,10.819201,1.276991,68.39156,12.9,10022.5670,9.212595,2.011,48.964695
20,1.0,AUS,2017,10.946128,0.85,0.7225,0.614125,15.124,228.735376,3459.393827,54027.966818,10.897257,1.948647,65.60463,12.9,10238.4130,9.233902,2.011,49.731342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,1.0,LTU,2014,14.185811,0.43,0.1849,0.079507,10.233,104.714289,1071.541319,16564.956549,9.715045,0.103758,52.58745,13.0,5288.8008,8.573347,2.697,11.871613
691,1.0,LTU,2015,13.743647,0.53,0.2809,0.148877,10.256,105.185536,1078.782857,14258.229335,9.565090,-0.884097,53.33852,13.0,5523.3896,8.616747,2.697,8.560302
692,1.0,LTU,2016,11.403278,0.51,0.2601,0.132651,10.470,109.620900,1147.730823,14998.125060,9.615680,0.905525,50.88945,13.0,6170.3911,8.727518,2.697,11.594822
693,1.0,LTU,2017,11.220660,0.54,0.2916,0.157464,10.619,112.763161,1197.432007,16885.407395,9.734205,3.722889,47.00182,13.0,6339.7036,8.754587,2.697,12.785031


In [31]:
df.to_csv('./data/panel_data/Full_DB.csv',index=False)